In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
from app import zillowELT,dfs_shape_merge,google_zip_df,merge_dfs,regression

In [110]:
df = pd.read_sql("regression",con_sum)

In [113]:
df.drop(columns=["rent"]).dropna()

,zip,sales,roi,population,density,violent crime,property crime,RESTAURANT_count,RESTAURANT_mean
1,94103,1104.937460,4.703656,23016.0,17140.46,65.9,87.6,20.0,683.60
2,94105,1442.732210,4.033728,2058.0,5902.54,33.7,70.0,20.0,806.15
3,94107,1076.296165,4.729701,17368.0,7126.09,43.9,81.1,20.0,877.35
5,94109,1230.308724,4.849441,56322.0,46521.46,50.9,83.5,20.0,657.30
6,94110,1014.040239,4.617224,74633.0,31343.73,47.2,82.1,20.0,1012.10
8,94112,782.711435,4.689653,73104.0,22097.56,39.4,79.2,20.0,251.85
9,94114,1187.840039,4.240120,30574.0,22377.49,37.2,78.8,20.0,509.25
10,94115,1170.044469,4.447323,33115.0,29461.94,48.8,82.7,20.0,411.80
11,94116,915.099625,4.485435,42958.0,16629.92,33.8,67.7,20.0,234.15
12,94117,1047.916658,4.334088,38738.0,27861.23,40.3,80.4,20.0,585.55


In [99]:
# typ = request.form["typ"]
# srch = request.form["srch"]
# poi = request.form["pois"]
typ = "city"
srch = "san francisco"
poi = "starbucks,restaurant"

con_search = create_engine("sqlite:///search.sqlite")
df = pd.DataFrame({"Title":["Type","Search","Place of Interests"],"Input":[typ,srch,poi]})
df.to_sql("search", con_search, if_exists="replace", index=False)

typ = typ.lower()
#to take account typing differences such as space and capital letters
srch = srch.upper().replace(" ","").split(",")
poi = poi.upper()

#call in information to find what zipcodes are in search area(eg:chicago)
con_us = create_engine("sqlite:///us_db.sqlite")
city = pd.read_sql("city",con_us)
city = city[city[typ].astype(str).str.replace(" ","").str.upper().isin(srch)]
#from zip codes obtained, select information needed from each database
zips = list(city["zip"])

#         rent = pd.read_csv("https://raw.githubusercontent.com/yundk7/area_lookup_heroku/master/local/Zip_ZriPerSqft_AllHomes.csv")
rent = pd.read_sql("zillow_rent",con_us)
rent = zillowELT(rent,zips)
# rent_plt = zillowplot(rent.T)

#         sales = pd.read_csv("https://raw.githubusercontent.com/yundk7/area_lookup_heroku/master/local/Zip_MedianListingPricePerSqft_AllHomes.csv")
sales = pd.read_sql("zillow_sales",con_us)
sales = zillowELT(sales,zips)
# sales_plt = zillowplot(sales.T)

dfs = dfs_shape_merge(rent,sales)
rent1 = dfs[0]
sales1 = dfs[1]
ratio = (rent1 * 12) / sales1 * 100
# ratio_plt = zillowplot(ratio.T)

In [101]:
len(sales)

19

In [103]:
# getting recent 5 for regression later
n = -5
rent = rent.iloc[:,n:]
rent = pd.DataFrame({"rent":rent.mean(axis=1)})

sales = sales.iloc[:,n:]
sales = pd.DataFrame({"sales":sales.mean(axis=1)})

ratio = ratio.iloc[:,n:]
ratio = pd.DataFrame({"roi":ratio.mean(axis=1)})

In [104]:
# now the zips are outer merge of zips in rent and sales
zip_rent = pd.DataFrame(rent.index.values)
zip_sales = pd.DataFrame(sales.index.values)
zips = list(pd.merge(zip_rent,zip_sales,on=0,how="outer")[0])

In [105]:
len(zips)

24

In [106]:
#preparing dataframe for regression with zipcode data, also to be used in google api search
area = pd.read_sql("area",con_us)
area = area[area["zip"].isin(zips)]
crime = pd.read_sql("crime",con_us)

regr = pd.merge(area,crime,on="zip")
regr.set_index("zip",inplace=True)

In [107]:
len(regr)

23

In [76]:
#since heroku is limited with request time, sampling out 8 zip codes to analyze
# sample = 8
# if len(df) < sample:
#     sample = len(df)
# df = df.sample(sample)

#using regression dataframe for reference
api = google_zip_df(regr,poi)
# geo_plt = plotly_geo(api)
# plotly_geo(api)
con_sum = create_engine("sqlite:///summary.sqlite")
api.to_sql("api",con_sum,if_exists="replace",index=False)

#preparing api results for regression
API_df = api[api["poi"]!="YOU ARE HERE"]
API_df[["reviews"]]=API_df[["reviews"]].apply(pd.to_numeric, errors='coerce')
count_df = API_df.pivot_table(fill_value=0,index = "zip",columns = ["poi"], values="reviews",aggfunc=["count"])["count"].reset_index()
mean_df = API_df.pivot_table(fill_value=0,index = "zip",columns = ["poi"], values="reviews",aggfunc=["mean"])["mean"].reset_index()
API_pivot = pd.merge(count_df,mean_df,on="zip",suffixes=["_count","_mean"])
API_pivot.set_index("zip",inplace=True)

regr = merge_dfs([regr,API_pivot])

regr.drop(columns = ["coordinates","area","radius"],inplace=True)

#regression on each
rent = merge_dfs([rent,regr])
rent = regression(rent)
rent[0].to_sql("rent0",con_sum,if_exists="replace",index=False)
rent[1].to_sql("rent1",con_sum,if_exists="replace",index=True)

sales = merge_dfs([sales,regr])
sales = regression(sales)
sales[0].to_sql("sales0",con_sum,if_exists="replace",index=False)
sales[1].to_sql("sales1",con_sum,if_exists="replace",index=True)

ratio = merge_dfs([ratio,regr])
ratio = regression(ratio)
ratio[0].to_sql("ratio0",con_sum,if_exists="replace",index=False)
ratio[1].to_sql("ratio1",con_sum,if_exists="replace",index=True)

In [109]:
len(regr)

23